In [2]:
#import libraries
import pandas as pd
import numpy as np

In [21]:
add = 'https://raw.githubusercontent.com/WalePhenomenon/climate_change/master/fuel_ferc1.csv'
df = pd.read_csv(add)

In [22]:
#df

Replacing blanks with numpy's nan, if any

In [23]:
df = df.replace(r'^\s+$', np.nan, regex=True)

In [24]:
df.describe()

,utility_id_ferc1,report_year,fuel_qty_burned,fuel_mmbtu_per_unit,fuel_cost_per_unit_burned,fuel_cost_per_unit_delivered,fuel_cost_per_mmbtu
count,29523.000000,29523.000000,2.952300e+04,29523.000000,29523.000000,2.952300e+04,29523.000000
mean,118.601836,2005.806050,2.622119e+06,8.492111,208.649031,9.175704e+02,19.304354
std,74.178353,7.025483,9.118004e+06,10.600220,2854.490090,6.877593e+04,2091.540939
min,1.000000,1994.000000,1.000000e+00,0.000001,-276.080000,-8.749370e+02,-41.501000
25%,55.000000,2000.000000,1.381700e+04,1.024000,5.207000,3.778500e+00,1.940000
50%,122.000000,2006.000000,2.533220e+05,5.762694,26.000000,1.737100e+01,4.127000
75%,176.000000,2012.000000,1.424034e+06,17.006000,47.113000,4.213700e+01,7.745000
max,514.000000,2018.000000,5.558942e+08,341.260000,139358.000000,7.964521e+06,359278.000000


check columns with Nan

In [25]:
#this line of code will be used to check each column for missing values 
for col in df.columns:
    print(f'{col} : {df[col].isnull().values.any()}')

record_id : False
utility_id_ferc1 : False
report_year : False
plant_name_ferc1 : False
fuel_type_code_pudl : False
fuel_unit : True
fuel_qty_burned : False
fuel_mmbtu_per_unit : False
fuel_cost_per_unit_burned : False
fuel_cost_per_unit_delivered : False
fuel_cost_per_mmbtu : False


Knowing how many nans we have 

In [26]:
nans = df.isnull()
for col in nans.columns.values.tolist():
    print(col)
    print (nans[col].value_counts())
    print("") 

record_id
False    29523
Name: record_id, dtype: int64

utility_id_ferc1
False    29523
Name: utility_id_ferc1, dtype: int64

report_year
False    29523
Name: report_year, dtype: int64

plant_name_ferc1
False    29523
Name: plant_name_ferc1, dtype: int64

fuel_type_code_pudl
False    29523
Name: fuel_type_code_pudl, dtype: int64

fuel_unit
False    29343
True       180
Name: fuel_unit, dtype: int64

fuel_qty_burned
False    29523
Name: fuel_qty_burned, dtype: int64

fuel_mmbtu_per_unit
False    29523
Name: fuel_mmbtu_per_unit, dtype: int64

fuel_cost_per_unit_burned
False    29523
Name: fuel_cost_per_unit_burned, dtype: int64

fuel_cost_per_unit_delivered
False    29523
Name: fuel_cost_per_unit_delivered, dtype: int64

fuel_cost_per_mmbtu
False    29523
Name: fuel_cost_per_mmbtu, dtype: int64



#Dealing with column Fuel unit 
    1. it should be an object and not integer (convert to obj)
    2.140 of 29523 is a quite small (dropping all may be considered )
    3. would investigate dataset for significance of the intended  drops on the dataset 
        a. qr grid or excel would be used for this 

In [9]:
# import qgrid as qr
# qrw = qr.show_grid(df,show_toolbar=True)
# qrw

Having investigated the dataset:\
    1. It turns out  elements from column(fuel_type) can be used to ascertain fuel unit\
    2. However, these fuel type have numerous fuel unit, and since we are dealing with measures of volume , assigning wrong unit to a fuel type could be catastophic to our analysis \
    3. The "waste" fuel type however can be salvaged , since 99.9% of its fuel units are in ton and its just one row that has bbl has unit. \
    4. So its only the Nan with fuel type 'Waste' that will be be replaced using measure of central tendencies Mode \
    5.Every other Nan will be dropped 

In [27]:
#This code replaces Nans in fuel_unit where where Fuel_type == Waste
df_fuel_waste = df[df['fuel_type_code_pudl'] == 'waste']
df['fuel_unit'].fillna(df_fuel_waste['fuel_unit'].replace(np.nan,'ton'), inplace=True)

In [28]:
#confirm the replacement worked
nans = df.isnull()
for col in nans.columns.values.tolist():
    print(col)
    print (nans[col].value_counts())
    print("") 

record_id
False    29523
Name: record_id, dtype: int64

utility_id_ferc1
False    29523
Name: utility_id_ferc1, dtype: int64

report_year
False    29523
Name: report_year, dtype: int64

plant_name_ferc1
False    29523
Name: plant_name_ferc1, dtype: int64

fuel_type_code_pudl
False    29523
Name: fuel_type_code_pudl, dtype: int64

fuel_unit
False    29347
True       176
Name: fuel_unit, dtype: int64

fuel_qty_burned
False    29523
Name: fuel_qty_burned, dtype: int64

fuel_mmbtu_per_unit
False    29523
Name: fuel_mmbtu_per_unit, dtype: int64

fuel_cost_per_unit_burned
False    29523
Name: fuel_cost_per_unit_burned, dtype: int64

fuel_cost_per_unit_delivered
False    29523
Name: fuel_cost_per_unit_delivered, dtype: int64

fuel_cost_per_mmbtu
False    29523
Name: fuel_cost_per_mmbtu, dtype: int64



In [12]:
import qgrid as qr
qrw = qr.show_grid(df,show_toolbar=True)


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

Drop other Nans

In [17]:
#df=df['fuel_unit'].dropna(axis=0,how='all')